<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1>Elementos interactivos para consulta en datos abiertos
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.colombiacompra.gov.co/sites/cce_public/files/logo_dnp.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Octubre 2022|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Elementos interactivos para consulta en datos abiertos**|
|**Descripción y alcance**|Script para desarrollar elementos interactivos para consulta en datos abiertos.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>Nombre:</td> <td>Equipo analítica EMAE</td></tr> <tr><td>Cargo:</td> <td>NA</td></tr>  <tr><td>Área:</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>Nombre:</td> <td>Carlos Isaac Zainea Maya</td></tr> <tr><td>Cargo:</td> <td>Contratista</td></tr>  <tr><td>Área:</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>Nombre:</td> <td>Catalina Pimienta</td></tr> <tr><td>Cargo:</td> <td>Subdirectora Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import dash_table
from datetime import date

from col_consulta import *
from col_credentials import *

In [6]:
#df=consulta_mae('Bogotá DC','AGENCIA NACIONAL DE INFRAESTRUCTURA ANI','2010-01-01','2022-12-31','0','100000000000000000','0')
#df

In [7]:
def consulta_MAE_simple():
    clas_unspsc=pd.read_csv('datos/clasificador_de_bienes_y_servicios.csv')
    clas_unspsc=clas_unspsc[['Nombre Clase','Código Clase']].drop_duplicates()
    clas_unspsc.set_index('Código Clase',inplace=True)
    app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    df=pd.read_csv('datos/contratos_entidad.csv')
    app.layout=html.Div([
        html.H1("Análisis de datos del sistema de compra pública"),
        html.H2("Datos de la compra pública"),
        html.H3("Seleccione el departamento"),
        dcc.Dropdown(
            id='departamento',
            options=[{'label': i, 'value': i} for i in dptos],
            multi=True,
            value='Cundinamarca'),
        html.H3("Seleccione el año"),
        dcc.Dropdown(
            id='anio',
            options=[{'label': i, 'value': i} for i in ['2018','2019','2020','2021','2022']],
            multi=True,
            value='2022'),
        html.H3("Seleccione la entidad"),
        dcc.Dropdown(
            id='dropdown_entidad',
            multi=True,
            value='Todas',
            ),
        
        dbc.Row([
            
            dbc.Col([
            html.Div(id='card_1'),
            html.Div(id='card_2'),
            html.Div(id='card_3'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_1'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_2'),
            ]),    

    ])
        
    ])
    @app.callback(Output('dropdown_entidad','options'),
                [Input('departamento','value'),
                Input('anio','value'),])
    def update_dropdown_entidad(departamento,anio):
        if departamento is None:
            departamento=dptos
        if anio is None:
            anio=2022
        if type(departamento)==str:
            departamento=[departamento]
        if type(anio)==str:
            anio=[anio]
        lista_consulta=[]
        for dpto in departamento:
            for a in anio:
                a=int(a)
                df_dpto=consulta_datos_departamento(dpto,a)
                lista_consulta.append(df_dpto)
        df=pd.concat(lista_consulta)
        return [{'label': i, 'value': i} for i in list(df['entidad'].unique())+['Todas']]


    @app.callback(
        Output('card_1', 'children'),
        Output('card_2', 'children'),
        Output('card_3', 'children'),
        Output('grafica_1', 'figure'),
        Output('grafica_2', 'figure'),
        Input('departamento', 'value'),
        Input('anio', 'value'),
        Input('dropdown_entidad', 'value'),
        )
    def update_output(departamento,anio,entidad):
        if type(departamento)==str:
            departamento=[departamento]
        if type(anio)==str:
            anio=[anio]
        lista_consulta=[]
        for dpto in departamento:
            for an in anio:
                an=int(an)
                df_temp=consulta_datos_departamento(dpto,an)
                lista_consulta.append(df_temp)
        df=pd.concat(lista_consulta)
        if type(entidad)==str and entidad!='Todas':
            entidad=[entidad]
            df=df[df['entidad'].isin(entidad)]
        if type(entidad)==str and entidad=='Todas':
            None
        else:
            df=df[df['entidad'].isin(entidad)]
        df=df.dropna()
        df=df.drop_duplicates()
        table=dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
        )
        card_1=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Número de contratos", className="card-title"),
                    html.P(
                        df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_2=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Valor total de contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_3=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Promedio valor de Contratos", className="card-title"),
                    html.P(
                        df['suma_valor_contrato'].sum()/df['cantidad'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        ),
        fig1=px.bar(df, x="año", y="suma_valor_contrato", color="departamento_entidad")
        fig2=px.bar(df, x="año", y="cantidad",  color="departamento_entidad")
        return card_1,card_2,card_3,fig1,fig2#,table

    return app.run_server(mode='inline',debug=True)

In [8]:
def consulta_MAE(port=np.random.randint(1000,9999)):
    app=JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
    clas_unspsc=pd.read_csv('datos/clasificador_de_bienes_y_servicios.csv')
    clas_unspsc=clas_unspsc[['Nombre Clase','Código Clase']].drop_duplicates()
    clas_unspsc.set_index('Código Clase',inplace=True)
    df=pd.read_csv('datos/contratos_entidad.csv')
    app.layout=html.Div([
        html.H1("Análisis de datos del sistema de compra pública"),
        html.H2("Datos de la compra pública"),
        dbc.Row([            
            dbc.Col([
        html.H3("Seleccione el departamento"),
        dcc.Dropdown(
            id='departamento',
            options=[{'label': i, 'value': i} for i in dptos],
            multi=True,
            value='Cundinamarca'),
            ]),         
            dbc.Col([
        html.H3("Seleccione el valor inicial del rango"),
        dcc.Input(
            id='valor_inicial',
            type='text',
            value='0'),
            ]),
        dbc.Col([
        html.H3("Seleccione el valor final del rango"),
        dcc.Input(
            id='valor_final',
            type='text',
            value='1000000000'),

            ]),
        ])    ,
        dbc.Row([            
            dbc.Col([
        html.H3("Seleccione rango de fechas"),
        dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=date(2000, 1, 1),
        max_date_allowed=date(2023, 1, 1),
        start_date=date(2010,1, 1),
        end_date=date(2022, 9, 30)
    )

            ]),
            dbc.Col([
        html.H3("Seleccione Clase UNSPSC"),
        dcc.Dropdown(
            id='unspsc',
            multi=True,
            options=[{'label': str(i)+' - '+clas_unspsc['Nombre Clase'][i], 'value': str(i)} for i in clas_unspsc.index],
            ),
            ]),
    ]),
        dbc.Row([
                dbc.Col([
        html.H3("Seleccione la entidad"),
        dcc.Dropdown(
            id='dropdown_entidad',
            multi=True,
            value='ALCALDIA ZIPACON',
            ),
            ]),
               ]),
        html.Button("Consultar y descargar CSV", id="btn_csv"),
        dcc.Download(id="download-dataframe-csv"),
        dbc.Row([
            
            dbc.Col([
            html.Div(id='card_1'),
            html.Div(id='card_2'),
            html.Div(id='card_3'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_1'),
            ]),
            dbc.Col([
            dcc.Graph(id='grafica_2'),
            ]),    

    ])
        
    ])
    @app.callback(Output('dropdown_entidad','options'),
                [Input('departamento','value'),])
    def update_dropdown_entidad(departamento):
        anio=[str(i) for i in range(2000,2023)]
        if departamento is None:
            departamento=dptos
        if type(departamento)==str:
            departamento=[departamento]
        lista_consulta=[]
        for dpto in departamento:
            for a in anio:
                a=int(a)
                df_dpto=consulta_datos_departamento(dpto,a)
                lista_consulta.append(df_dpto)
        df=pd.concat(lista_consulta)
        return [{'label': i, 'value': i} for i in list(df['entidad'].unique())+['Todas']]


    @app.callback(
        Output('card_1', 'children'),
        Output('card_2', 'children'),
        Output('card_3', 'children'),
        Output('grafica_1', 'figure'),
        Output('grafica_2', 'figure'),
        Output("download-dataframe-csv", "data"),
        Input('departamento', 'value'),
        Input('dropdown_entidad', 'value'),
        Input('valor_inicial', 'value'),
        Input('valor_final', 'value'),
        Input('unspsc', 'value'),
        Input('my-date-picker-range', 'start_date'),
        Input('my-date-picker-range', 'end_date'),
        Input("btn_csv", "n_clicks"),prevent_initial_call=True,
        )
    def update_output(departamento,entidad,valor_inicial,valor_final,unspsc,fecha_inicio,fecha_fin,n_clicks):
        valor_inicial=str(valor_inicial)
        valor_final=str(valor_final)
        fecha_inicio_obj = date.fromisoformat(fecha_inicio)
        fecha_inicio_str = fecha_inicio_obj.strftime('%Y-%m-%d')
        fecha_fin_obj = date.fromisoformat(fecha_inicio)
        fecha_fin_str = fecha_fin_obj.strftime('%Y-%m-%d')
        if type(departamento)==str:
            departamento=[departamento]
        if type(entidad)==str:
            entidad=[entidad]
        if type(unspsc)==str:
            unspsc=[unspsc]      
        if unspsc==None:
            unspsc=['0']  
        lista_consulta=[]


        for dpto in departamento:
            for ent in entidad:
                for u in unspsc:
                    #print(dpto+ "|"+ ent+ "|"+ fecha_inicio+ "|"+ fecha_fin+ "|"+ valor_inicial+ "|"+ valor_final+ "|"+ u)
                    try:
                        
                        df_dpto=consulta_mae(dpto,ent,fecha_inicio,fecha_fin,valor_inicial,valor_final,u)
                        lista_consulta.append(df_dpto)
                    except:
                        None
                        continue
        print(lista_consulta)
        global df
        df=pd.concat(lista_consulta)
        df=df.dropna()
        df=df.drop_duplicates()
        card_1=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Número de contratos", className="card-title"),
                    html.P(
                        df.shape,
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_2=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Valor total de contratos", className="card-title"),
                    html.P(
                        df['valor_contrato'].sum(),
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        )
        card_3=dbc.Card(
            dbc.CardBody(
                [
                    html.H4("Promedio valor de Contratos", className="card-title"),
                    html.P(
                        df['valor_contrato'].sum()/df.shape[0],
                        className="card-text",
                    ),
                ]
            ),
            style={"width": "18rem"},
        ),
        fig1=px.pie(df, values='valor_contrato', names='departamento_entidad', title='Valor de contratos por departamento')
        fig2=px.bar(df, x="modalidad", y="valor_contrato",  color="fuente")
        return card_1,card_2,card_3,fig1,fig2, dcc.send_data_frame(df.to_csv, "mydf.csv")#,table


    return app.run_server(mode='inline',debug=True,port=port)

In [9]:
consulta_MAE()

[]
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File c:\Users\cizai\miniconda3\lib\site-packages\pandas\util\_decorators.py:311, in deprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper(*args=([],), **kwargs={})
    305 if len(args) > num_allow_args:
    306     warnings.warn(
    307         msg.format(arguments=arguments),
    308         FutureWarning,
    309         stacklevel=stacklevel,
    310     )
--> 311 return func(*args, **kwargs)
        func = <function concat at 0x000001F5F20D4160>
        args = ([],)
        kwargs = {}

File c:\Users\cizai\miniconda3\lib\site-packages\pandas\core\reshape\concat.py:347, in concat(
    objs=[],
    axis=0,
    join='outer',
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    sort=False,
    copy=True
)
    143 @deprecate_nonkeyword_arguments(version=None, allowed_args

## Aprobación
|||
|--|--|
|**Nombre**|Wilson Camilo Sanchez|
|**Cargo**|Gestor T11|
|**Área**|Subdirección EMAE|

![version](https://github.com/ANCP-CCE-Analitica/datos_abiertos/blob/main/images/version.png?raw=1)